# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [32]:
# initial imports
import os
import pandas as pd
import panel as pn
import plotly.express as px
from pathlib import Path
from dotenv import load_dotenv

In [33]:
# Initialize the Panel Extensions (for Plotly)
pn.extension('plotly')

In [34]:
import hvplot.pandas

In [35]:
import matplotlib.pyplot as plt

In [36]:
# Read the Mapbox API key
load_dotenv()
mapbox_token = os.getenv("mapbox")
px.set_mapbox_access_token(mapbox_token)

# Import Data

In [37]:
# Import the CSVs to Pandas DataFrames
sfo_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(sfo_path).drop_duplicates()

neighborhood_path = Path("Data/neighborhoods_coordinates.csv")
neighborhood_locations = pd.read_csv(neighborhood_path).drop_duplicates()

In [38]:
neighborhood_data = (
    pd.concat([sfo_data, neighborhood_locations], axis=1, sort=True)
    .dropna()
    .reset_index()
)
neighborhood_data.head()

,index,year,neighborhood,sale_price_sqr_foot,housing_units,gross_rent,Neighborhood,Lat,Lon
0,0,2010,Alamo Square,291.182945,372560,1239,Alamo Square,37.791012,-122.402100
1,1,2010,Anza Vista,267.932583,372560,1239,Anza Vista,37.779598,-122.443451
2,2,2010,Bayview,170.098665,372560,1239,Bayview,37.734670,-122.401060
3,3,2010,Buena Vista Park,347.394919,372560,1239,Bayview Heights,37.728740,-122.410980
4,4,2010,Central Richmond,319.027623,372560,1239,Bernal Heights,37.728630,-122.443050


In [39]:
neighborhood_data = neighborhood_data.drop(columns="neighborhood")
neighborhood_data.head()

,index,year,sale_price_sqr_foot,housing_units,gross_rent,Neighborhood,Lat,Lon
0,0,2010,291.182945,372560,1239,Alamo Square,37.791012,-122.402100
1,1,2010,267.932583,372560,1239,Anza Vista,37.779598,-122.443451
2,2,2010,170.098665,372560,1239,Bayview,37.734670,-122.401060
3,3,2010,347.394919,372560,1239,Bayview Heights,37.728740,-122.410980
4,4,2010,319.027623,372560,1239,Bernal Heights,37.728630,-122.443050


In [40]:
neighborhood_index = neighborhood_data.set_index("year").drop(columns="index")
neighborhood_index.head()

,sale_price_sqr_foot,housing_units,gross_rent,Neighborhood,Lat,Lon
year,,,,,,
2010,291.182945,372560,1239,Alamo Square,37.791012,-122.402100
2010,267.932583,372560,1239,Anza Vista,37.779598,-122.443451
2010,170.098665,372560,1239,Bayview,37.734670,-122.401060
2010,347.394919,372560,1239,Bayview Heights,37.728740,-122.410980
2010,319.027623,372560,1239,Bernal Heights,37.728630,-122.443050


In [41]:
neighborhood_index = neighborhood_index.rename(
    columns={"sale_price_sqr_foot": "Price per Sqr Foot",
            "housing_units": "Housing Unites",
            "gross_rent": "Gross Rent",
            "Lat": "Latitude",
            "Lon": "Longitude"})
neighborhood_index.head()

,Price per Sqr Foot,Housing Unites,Gross Rent,Neighborhood,Latitude,Longitude
year,,,,,,
2010,291.182945,372560,1239,Alamo Square,37.791012,-122.402100
2010,267.932583,372560,1239,Anza Vista,37.779598,-122.443451
2010,170.098665,372560,1239,Bayview,37.734670,-122.401060
2010,347.394919,372560,1239,Bayview Heights,37.728740,-122.410980
2010,319.027623,372560,1239,Bernal Heights,37.728630,-122.443050


- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [49]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
sfo_housing_units = sfo_data.groupby("year")["housing_units"].mean()
housing = px.bar(sfo_housing_units, range_y=("360000", "390000"),labels={"year": "Year", "value": "Housing Units"})
#return housing

def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
sfo_avg_gross_rent = sfo_data.groupby("year")["sale_price_sqr_foot", "gross_rent"].mean()
sfo_gross = px.line(sfo_avg_gross_rent, labels={"year": "Year"})
#return sfo_gross

def average_sales_price():
    """Average Sales Price Per Year."""
sfo_sale_sqr_foot = sfo_data.groupby("year")["sale_price_sqr_foot"].mean()
sfo_sale = px.line(sfo_sale_sqr_foot, labels={"year": "Year", "value": "Housing Units"})
#return sfo_sale

def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
sfo_hood_avg = sfo_data.groupby(["year", "neighborhood"]).mean().reset_index()
sfo_hood_avg_plot = sfo_hood_avg.hvplot.line(x="year", y="sale_price_sqr_foot",
                                            groupby="neighborhood",
                                            xlabel="Year", ylabel="Sales Price per Square Foot")
#return sfo_hood_avg_plot

def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
neighbor_data= sfo_data.groupby("neighborhood").mean().reset_index()
top_10_neighborhood = neighbor_data.nlargest(10,"sale_price_sqr_foot")
top_10 = top_10_neighborhood.hvplot.bar(x="neighborhood", y="sale_price_sqr_foot", 
                        xlabel ="Neighbornood", 
                        ylabel= "Avg. Sale Price per Square Foot",
                        title ="Top 10 Expensive Neighborhoods in SFO",rot=90)
#return top_10

def parallel_coordinates():
    """Parallel Coordinates Plot."""
coordinates = px.parallel_coordinates(top_10_neighborhood, color="sale_price_sqr_foot")
#return coordinates

def parallel_categories():
    """Parallel Categories Plot."""
categories = px.parallel_categories(
    top_10_neighborhood,
    dimensions=["neighborhood","sale_price_sqr_foot", "housing_units", "gross_rent"],
    color="sale_price_sqr_foot")
#return categories

def neighborhood_map():
    """Neighborhood Map"""    
map_1 = px.scatter_mapbox(
    neighborhood_index,
    lat="Latitude",
    lon="Longitude",
    size="Price per Sqr Foot",
    color="Neighborhood",
    title="Average Sales Price per Square Foot and Gross Rent in San Francisco")
#return map_1

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [50]:
text = "This dashboard presents a visual analysis of historical prices of house units, sale price per square foot and gross rent in San Francisco, California from 2010 to 2016.  You can navigate through the tabs above to explore more details about the evolution of the real estate market on The Golden City across these years."
welcome = pn.Column(text, map_1)

In [51]:
yma_rows = pn.Row(housing, sfo_gross, sfo_sale)

In [52]:
neighborhood_cols = pn.Column(sfo_hood_avg_plot, top_10)

In [53]:
parallel_plots = pn.Column(coordinates, categories)

In [54]:
# Create tabs
tabs = pn.Tabs(
    ("Welcome", welcome),
    ("Yearly Market Analysis", yma_rows),
    ("Neighborhood Analysis", neighborhood_cols),
    ("Parallel Plots Analysis", parallel_plots))
tabs

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Plotly(Figure)
    [1] Row
        [0] Plotly(Figure)
        [1] Plotly(Figure)
        [2] Plotly(Figure)
    [2] Column
        [0] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()
        [1] HoloViews(Bars)
    [3] Column
        [0] Plotly(Figure)
        [1] Plotly(Figure)

In [ ]:
# Group by year and neighborhood and then create a new dataframe of the mean values
sfo_hood= sfo_data.groupby(["neighborhood","year"]).mean()

In [ ]:
# Slice for neighborhood
neighbor_group= sfo_data.groupby("neighborhood").mean().reset_index()

## Serve the Panel Dashboard

In [ ]:
panel.servable()